<a href="https://colab.research.google.com/github/tnfru/colab_research/blob/master/CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR100
import torchvision.models
from torchvision import transforms as T
from torch.utils.data import DataLoader
from torchsummary import summary
import time

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device:', DEVICE)
torch.backends.cudnn.benchmark = True

using device: cuda


In [3]:
BATCH_SIZE = 128
IMG_SIZE = [32, 32]
TRAIN_MEANS = np.array([0.50707516, 0.48654887, 0.44091784])
TRAIN_STDS = np.array([0.26733429, 0.25643846, 0.27615047])
TEST_MEANS = np.array([0.50879641, 0.48739301, 0.44194221])
TEST_STDS = np.array([0.26825157, 0.25736374, 0.27709577])

In [4]:
tr_train = T.Compose([T.ToTensor(),
                      T.RandomHorizontalFlip(p=0.5),
                      T.Normalize(TRAIN_MEANS, TRAIN_STDS)])

tr_test = T.Compose([T.ToTensor(),
                     T.Normalize(TEST_MEANS, TEST_STDS)])

train = CIFAR100("./data", train=True, download=True, transform=tr_train)
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

test = CIFAR100("./data", train=False, download=True, transform=tr_test)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [42]:
class IdentityModule(nn.Module):
  def __init__(self, num_channel):
    super(IdentityModule, self).__init__()

    self.hidden1 = nn.Sequential(
        nn.Conv2d(num_channel, num_channel, 3, padding=1),
        nn.BatchNorm2d(num_channel),
        nn.ReLU(),
        nn.Conv2d(num_channel, num_channel, 3, padding=1),
        nn.BatchNorm2d(num_channel),
        nn.ReLU()
    )

    self.hidden2 = nn.Sequential(
        nn.Conv2d(num_channel, num_channel, 3, padding=1),
        nn.BatchNorm2d(num_channel),
        nn.ReLU(),
        nn.Conv2d(num_channel, num_channel, 3, padding=1),
        nn.BatchNorm2d(num_channel),
        nn.ReLU()
    )
  
  def forward(self, x): # TODO swap Relu after add
    y1 = self.hidden1(x)
    y2 = self.hidden2(x)

    return x + y1 + y2

In [43]:
class ConvModule(nn.Module):
  def __init__(self, in_channel, out_channel, stride=2):
    super(ConvModule, self).__init__()

    self.hidden1 = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, 3, padding=1, stride=stride),
        nn.BatchNorm2d(out_channel),
        nn.ReLU(),
        nn.Conv2d(out_channel, out_channel, 3, padding=1),
        nn.BatchNorm2d(out_channel),
        nn.ReLU()
    )
    self.hidden2 = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, 3, padding=1, stride=stride),
        nn.BatchNorm2d(out_channel),
        nn.ReLU(),
        nn.Conv2d(out_channel, out_channel, 3, padding=1),
        nn.BatchNorm2d(out_channel),
        nn.ReLU()
    )
    self.hidden3 = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, 1, stride=stride),
        nn.BatchNorm2d(out_channel)
    )

  def forward(self, x):
    y1 = self.hidden1(x)
    y2 = self.hidden2(x)
    y3 = self.hidden3(x)

    return self.hidden1(x) + self.hidden2(x) + self.hidden3(x)


In [44]:
class ResNet(nn.Module):
  def __init__(self):
    super(ResNet, self).__init__()
    self.inp = nn.Sequential(
        nn.Conv2d(3, 16, 3, padding=1),
        nn.Dropout(0.1)
    )

    self.hidden1 = nn.Sequential(
        ConvModule(16, 64, stride=1),
        IdentityModule(64),
        nn.Dropout(0.15)
    )

    self.hidden2 = nn.Sequential(
        ConvModule(64, 128),
        IdentityModule(128),
        nn.Dropout(0.2)
    )

    self.hidden3 = nn.Sequential(
        ConvModule(128, 128),
        IdentityModule(128),
        nn.Dropout(0.25)
    )

    self.out = nn.Sequential(
        nn.AvgPool2d(kernel_size=8),
        nn.Flatten(),
        nn.Linear(128, 100),
        nn.Softmax(dim=1)
    )

  def forward(self, x):
    x = self.inp(x)
    x = self.hidden1(x)
    x = self.hidden2(x)
    x = self.hidden3(x)
    x = self.out(x)

    return x

In [45]:
def test_accuracy(model, history):
  model.eval()
  with torch.no_grad():
      correct = 0
      total = 0
      total_loss =0

      for data, target in test_loader:
          images = data.to(DEVICE)
          labels = target.to(DEVICE)
          outputs = model(images)

          _, predicted = torch.max(outputs, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

          accuracy = 100 * correct / total
          history.append(accuracy)

  model.train()

  return history

In [46]:
def train_model(model, opt, scheduler, num_epochs, history=[], verbose=True):
  criterion = nn.CrossEntropyLoss()

  for epoch in range(num_epochs):
    start_time = time.time()
    for i, (images, labels) in enumerate(train_loader):
      # Write tensor to GPU if available
      images, labels = images.to(DEVICE), labels.to(DEVICE)

      # Clear Gradient
      opt.zero_grad()

      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)

      # Backward pass
      loss.backward()
      opt.step()

    history = test_accuracy(model, history)

    if verbose:
      _, predicted = torch.max(outputs, 1)
      total = labels.size(0)
      correct = (predicted == labels).sum().item()

      train_accuracy = 100 * correct / total
      print('EPOCH: {} LR: {} Time: {:.4} sec Train Accuracy: {} % Test Accuracy: {} %'.format(epoch, 
            scheduler.get_last_lr(), time.time() - start_time, train_accuracy, history[-1]))
    
    scheduler.step()

  return history

In [55]:
model = ResNet()
model.to(DEVICE)
opt = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.MultiStepLR(opt, milestones=[100, 125], gamma=0.1)
history = []

In [56]:
history = train_model(model, opt, scheduler, num_epochs=150, history=history)

EPOCH: 0 LR: [0.001] Time: 66.72 sec Train Accuracy: 6.25 % Test Accuracy: 5.48 %
EPOCH: 1 LR: [0.001] Time: 66.12 sec Train Accuracy: 6.25 % Test Accuracy: 8.08 %
EPOCH: 2 LR: [0.001] Time: 65.98 sec Train Accuracy: 10.0 % Test Accuracy: 9.04 %
EPOCH: 3 LR: [0.001] Time: 66.28 sec Train Accuracy: 7.5 % Test Accuracy: 10.77 %
EPOCH: 4 LR: [0.001] Time: 66.25 sec Train Accuracy: 8.75 % Test Accuracy: 11.88 %
EPOCH: 5 LR: [0.001] Time: 66.24 sec Train Accuracy: 13.75 % Test Accuracy: 12.61 %
EPOCH: 6 LR: [0.001] Time: 66.19 sec Train Accuracy: 11.25 % Test Accuracy: 12.88 %
EPOCH: 7 LR: [0.001] Time: 66.4 sec Train Accuracy: 11.25 % Test Accuracy: 13.78 %
EPOCH: 8 LR: [0.001] Time: 66.05 sec Train Accuracy: 11.25 % Test Accuracy: 14.45 %
EPOCH: 9 LR: [0.001] Time: 66.17 sec Train Accuracy: 15.0 % Test Accuracy: 15.68 %
EPOCH: 10 LR: [0.001] Time: 66.35 sec Train Accuracy: 15.0 % Test Accuracy: 15.75 %
EPOCH: 11 LR: [0.001] Time: 66.44 sec Train Accuracy: 16.25 % Test Accuracy: 16.31 %
EP

In [54]:
!nvidia-smi

Thu Oct  8 15:23:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    34W /  70W |   3879MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------